<a href="https://colab.research.google.com/github/Abdullah-Al-Arafat/CAP-5610-Machine-Learning/blob/master/ML_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CAP 5610: Machine Learning** 
Home work 2


In [1]:
import os
import numpy as np
from keras.datasets import cifar10
from keras import Sequential
from keras.utils import np_utils
from keras.optimizers import Adagrad
from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Flatten, Dropout, BatchNormalization 
from keras import regularizers
from keras.models import model_from_json
from random import shuffle

Using TensorFlow backend.


Preparation/preprocessing of train, test, and validation dataset 

In [0]:
(train_images, train_labels),(test_images, test_labels) = cifar10.load_data()

## Creating validation set from training dataset

indices = np.random.permutation(len(train_images))

val_indices = indices[0:2000]
train_indices = indices[2000:]

val_images, val_labels = train_images[val_indices], train_labels[val_indices]

Train_images, Train_labels = train_images[train_indices], train_labels[train_indices]


## Normalizing the Train, test and validation sets

X_train = (Train_images/255).astype('float32')
y_train = np_utils.to_categorical(Train_labels, 10)

X_val = (val_images/255).astype('float32')
y_val = np_utils.to_categorical(val_labels, 10)

X_test = (test_images/255).astype('float32')
y_test = np_utils.to_categorical(test_labels, 10)

In [29]:
X_train.shape

(48000, 32, 32, 3)

In [0]:
# Keras Parameters
batch_size = 32
nb_classes = 10
nb_epoch = 30
img_rows, img_col = 32, 32
img_channels = 3

## Weight Decay for regularizer
weight_decay = 1e-4

In [7]:

## Build the CNN model with keras

model = Sequential()
model.add(Conv2D(32, (5,5), strides = (1,1), padding='same', kernel_regularizer= regularizers.l2(weight_decay),
                 activation ='relu', input_shape =(img_rows, img_col, img_channels), data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(32, (3,3), strides =  (2,2), padding='same', activation='relu', kernel_regularizer= regularizers.l2(weight_decay)))
model.add(BatchNormalization())


model.add(Conv2D(48, (5,5), strides= (1,1),padding='same', activation='relu', kernel_regularizer= regularizers.l2(weight_decay)))
model.add(BatchNormalization())


model.add(Conv2D(64, (3,3), strides= (2,2), padding='same', kernel_regularizer= regularizers.l2(weight_decay), activation ='relu'))
model.add(BatchNormalization())
 
model.add(Conv2D(64, (3,3), strides= (2,2), padding='same', kernel_regularizer= regularizers.l2(weight_decay), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(nb_classes, activation='softmax'))

model.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        2432      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 48)        38448     
_________________________________________________________________
batch_normalization_3

In [0]:

# Initiate Adagrad/RMSprop optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [9]:
## Train the model and get the training and validation loss
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
                    verbose=1, validation_data=(X_val, y_val))


Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 2000 samples
Epoch 1/30
50000/50000 [==============================] - 42s 842us/step - loss: 1.7765 - acc: 0.3702 - val_loss: 1.6249 - val_acc: 0.4275
Epoch 2/30
50000/50000 [==============================] - 37s 746us/step - loss: 1.5167 - acc: 0.4681 - val_loss: 1.4448 - val_acc: 0.4950
Epoch 3/30
50000/50000 [==============================] - 38s 752us/step - loss: 1.4165 - acc: 0.5059 - val_loss: 1.3340 - val_acc: 0.5385
Epoch 4/30
50000/50000 [==============================] - 37s 748us/step - loss: 1.3374 - acc: 0.5370 - val_loss: 1.2277 - val_acc: 0.5780
Epoch 5/30
50000/50000 [==============================] - 36s 727us/step - loss: 1.2725 - acc: 0.5661 - val_loss: 1.2285 - val_acc: 0.5650
Epoch 6/30
50000/50000 [==============================] - 36s 724us/step - loss: 1.2237 - acc: 0.5864 - val_loss: 1.2099 - val_acc: 0.5955
Epoch 7/30
50000/50000 [==============================] - 36s 728us/s

In [10]:
## Score trained model.
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 233us/step
Test loss: 1.7795472106933594
Test accuracy: 0.573
